In [3]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
from functions_roman_rubin import sim_fit
current_path = os.path.dirname(os.getcwd())
# print(current_path)
i=18
model='USBL'
algo='TRF'
path_TRILEGAL_set= current_path+'/TRILEGAL/PB_planet_split_1.csv'
path_to_save_model= current_path+'/test_sim_fit/'
path_to_save_fit= current_path+'/test_sim_fit/'
path_ephemerides= current_path+'/ephemerides/Gaia.txt'
path_dataslice = current_path+'/opsims/baseline/dataSlice.npy'

In [4]:
sim_fit(i,model,algo,path_TRILEGAL_set,path_to_save_model,path_to_save_fit,path_ephemerides,path_dataslice)

/home/anibal/roman_rubin


/home/anibal/anaconda3/envs/pyLIMA_utd/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/home/anibal/anaconda3/envs/pyLIMA_utd/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 20736 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/home/anibal/anaconda3/envs/pyLIMA_utd/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 20736 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/home/anibal/anaconda3/envs/pyLIMA_utd/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/home/anibal/anaconda3/envs/pyLIMA_utd/lib/python3.11/site-packages/erfa/core.py:133: Erf

Parallax(Full) estimated for the telescope W149: SUCCESS
Parallax(Full) estimated for the telescope u: SUCCESS
Parallax(Full) estimated for the telescope g: SUCCESS
Parallax(Full) estimated for the telescope r: SUCCESS
Parallax(Full) estimated for the telescope i: SUCCESS
Parallax(Full) estimated for the telescope z: SUCCESS
Parallax(Full) estimated for the telescope y: SUCCESS
['third_caustic', [0, 0]]
A good event to fit
Save the data and fit with  TRF
saving...
check_event  : Everything looks fine...


/home/anibal/anaconda3/envs/pyLIMA_utd/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 20736 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/home/anibal/anaconda3/envs/pyLIMA_utd/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 20736 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)


Parallax(Full) estimated for the telescope Roman: SUCCESS
Parallax(Full) estimated for the telescope g: SUCCESS
Parallax(Full) estimated for the telescope r: SUCCESS
Parallax(Full) estimated for the telescope i: SUCCESS
Parallax(Full) estimated for the telescope z: SUCCESS
Parallax(Full) estimated for the telescope y: SUCCESS


/home/anibal/anaconda3/envs/pyLIMA_utd/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/home/anibal/anaconda3/envs/pyLIMA_utd/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/home/anibal/anaconda3/envs/pyLIMA_utd/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 96 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/home/anibal/anaconda3/envs/pyLIMA_utd/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 96 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/home/anibal/anaconda3/envs/pyLIMA_utd/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarn

initial_guess  : Initial parameters guess SUCCESS
Using guess:  [2462592.427461098, 0.1474368375383864, 351.03436333528964, 9.353665605734084e-05, 1.1478599290943858, 0.0004842690847154, 2.170221123244217, 0.0058858226795134, -0.0320152930579978, 108.98890894257437, 227.1020197745454, 20.29705972956781, 44.95377491290438, 80.48701079970978, 115.85956729838873, 71.81287251572661, 166.46665663900635, 90.42961050277304, 198.9413151678031, 146.31165327648137, 218.25714149463693]
fit  : Trust Region Reflective fit SUCCESS
best_model: [2462582.4274610984, 0.10495356310421913, 702.0687266705792, 0.00018706593514021636, 1.3185220306150873, 0.0009685381694307999, 1.2034950788588044, 0.011771645359026799, -0.014090980994503205, 70.91695583930412, 423.48879683529674, 13.359252475582124, 86.29854109960397, 52.70380636424485, 220.82061602730195, 47.09836633813141, 320.91976690342983, 59.38744027325328, 381.11972104444334, 67.50109862098031, 413.56572370454626] chi2 302735.3463566707
check_event  : 

In [4]:
os.listdir(path_to_save_model)

['Event_RR_18_TRF.npy',
 'Event_RR_0_TRF.npy',
 'Event_0.h5',
 'Event_Roman_18_TRF.npy',
 'Event_18.h5',
 'Event_Roman_0_TRF.npy']

In [8]:
from functions_roman_rubin import read_data
info_dataset, pyLIMA_parameters, bands = read_data(path_to_save_model+'/Event_18.h5')
print(info_dataset, pyLIMA_parameters, bands)

['18', '/home/anibal/roman_rubin/TRILEGAL/PB_planet_split_1.csv', ['third_caustic', [0, 0]]] {'alpha': 2.170221123244217, 'fblend_W149': 206.28179213395828, 'fblend_g': 100.45169468197552, 'fblend_i': 223.6983332752606, 'fblend_r': 69.10845649953637, 'fblend_u': 2.493525365117888, 'fblend_y': 87.62721758298721, 'fblend_z': 173.00884416162543, 'fsource_W149': 317.17398829050336, 'fsource_g': 114.33135273107756, 'fsource_i': 262.4849251897241, 'fsource_r': 380.0504336063954, 'fsource_u': 4.933245072029084, 'fsource_y': 131.55231680142592, 'fsource_z': 230.63654880597986, 'ftotal_W149': 523.4557804244616, 'ftotal_g': 214.78304741305308, 'ftotal_i': 486.1832584649847, 'ftotal_r': 449.1588901059318, 'ftotal_u': 7.426770437146972, 'ftotal_y': 219.17953438441313, 'ftotal_z': 403.6453929676053, 'gblend_W149': 0.6503742417395918, 'gblend_g': 0.8786014709215518, 'gblend_i': 0.852233068674597, 'gblend_r': 0.18184022537153455, 'gblend_u': 0.5054533737348428, 'gblend_y': 0.6661016674853225, 'gblend